# Documentation

# Sample

In [22]:
import marvin_classificacao_engine
from marvin_python_toolbox.common.data import MarvinData
import sys 
!{sys.executable} -m pip install nltk 
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install sklearn

In [23]:
from marvin_python_toolbox.common.data import MarvinData
import pandas as pd
import nltk
from nltk.corpus import stopwords

products_data = pd.read_csv(MarvinData.download_file("https://s3.amazonaws.com/automl-example/produtos.csv"), delimiter=';', encoding='utf-8')

# concatenando as colunas nome e descricao
products_data['informacao'] = products_data['nome'] + products_data['descricao']
# excluindo linhas com valor de informacao ou categoria NaN
products_data.dropna(subset=['informacao', 'categoria'], inplace=True)
products_data.drop(columns=['nome', 'descricao'], inplace=True)

stop_words=set(stopwords.words("portuguese"))
# transforma a string em caixa baixa e remove stopwords
products_data['sem_stopwords'] = products_data['informacao'].str.lower().apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

tokenizer = nltk.RegexpTokenizer(r"\w+")
products_data['tokens'] = products_data['sem_stopwords'].apply(tokenizer.tokenize) # aplica o regex tokenizer
products_data.drop(columns=['sem_stopwords','informacao'],inplace=True) # Exclui as colunas antigas

products_data["strings"]= products_data["tokens"].str.join(" ") # reunindo cada elemento da lista
products_data.head()
marvin_initial_dataset = products_data


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split( # Separação dos dados para teste e treino
    marvin_initial_dataset["strings"], 
    marvin_initial_dataset["categoria"], 
    test_size = 0.2, 
    random_state = 10
)
# CountVectorizer
vect = CountVectorizer()
vect.fit(marvin_initial_dataset["strings"])

marvin_dataset = { 
    "X_train": vect.transform(X_train),
    "X_test": vect.transform(X_test),
    "y_train": y_train,
    "y_test" : y_test,
    "vect": vect
}

In [25]:
from sklearn.naive_bayes import MultinomialNB


clf = MultinomialNB()
clf.fit(marvin_dataset["X_train"], marvin_dataset["y_train"])

marvin_model = {
    "clf" : clf,
    "vect": marvin_dataset["vect"]
}

In [26]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
# Multinomial NB
y_prediction = marvin_model["clf"].predict(marvin_dataset["X_test"])
accuracy = accuracy_score(y_prediction, marvin_dataset["y_test"])

marvin_metrics = accuracy 

In [3]:
input_message = ["Figura Transformers Prime War Deluxe - E9687 - Hasbro"]

In [4]:
print(input_message)
input_message = marvin_model["vect"].transform(input_message)

['Figura Transformers Prime War Deluxe - E9687 - Hasbro']


NameError: name 'marvin_model' is not defined

In [4]:
final_prediction = marvin_model["clf"].predict(input_message)[0]
print("Predicted value: " + final_prediction)

NameError: name 'marvin_model' is not defined